In [1]:
import os

In [2]:
%pwd

'c:\\Users\\dharm\\OneDrive\\Desktop\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\dharm\\OneDrive\\Desktop\\Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config  
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
tf.config.run_functions_eagerly(True)

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        # Save the configuration object which holds all training parameters
        self.config = config

    def get_base_model(self):
        # Load the base model that was saved after adding the classification head
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path, compile=False)

        # Recompile to reset optimizer for current model variables
        self.model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
        loss="binary_crossentropy",
        metrics=["accuracy"]
        )

    def train_valid_generator(self):
        # Common preprocessing: Normalize pixel values to [0, 1] and Split 20% of data for validation
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,  
            validation_split=0.20,   
        )

        # Image size and batch settings for the generator
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Resize image to (height, width)
            batch_size=self.config.params_batch_size,        # Number of images per batch
            interpolation="bilinear"                         # Interpolation used for resizing
        )

        # Validation generator without augmentation
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,   # Folder structure: class subdirs inside this
            subset="validation",                   # Use 20% for validation
            shuffle=False,                         # Keep order stable for evaluation
            class_mode="binary",                   # Classify images as either 0 or 1
            **dataflow_kwargs
        )

        # Training generator with optional data augmentation
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,              # Random rotation
                horizontal_flip=True,           # Flip images horizontally
                width_shift_range=0.2,          # Horizontal shift
                height_shift_range=0.2,         # Vertical shift
                shear_range=0.2,                # Shear transformation
                zoom_range=0.2,                 # Random zoom
                **datagenerator_kwargs
            )
        else:
            # Use same generator as validation if no augmentation is enabled
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,   # Same data directory
            subset="training",                     # Use 80% for training
            shuffle=True,                          # Shuffle for better learning
             class_mode="binary",                   # Classify images as either 0 or 1
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        # Save the complete model (architecture + weights)
        model.save(path)

    def train(self, callback_list: list):
        
        # Calculate the number of batches per epoch for both training and validation
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Train the model using the generators and provided callbacks
        self.model.fit(
            self.train_generator,                   # Training generator
            epochs=self.config.params_epochs,       # Total number of epochs to train
            steps_per_epoch=self.steps_per_epoch,   # How many batches per epoch
            validation_steps=self.validation_steps, # Validation steps per epoch
            validation_data=self.valid_generator,   # Validation generator
            callbacks=callback_list                 # List of Keras callbacks (e.g., early stopping)
        )

        # Save the fully trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2025-05-22 00:57:09,745: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-22 00:57:09,749: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-22 00:57:09,751: INFO: common: created directory at: artifacts]
[2025-05-22 00:57:09,753: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-05-22 00:57:09,755: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-05-22 00:57:09,759: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


c:\Users\dharm\OneDrive\Desktop\Chicken-Disease-Classification\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
c:\Users\dharm\OneDrive\Desktop\Chicken-Disease-Classification\myenv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.5264 - loss: 5.5469 

c:\Users\dharm\OneDrive\Desktop\Chicken-Disease-Classification\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 218s 12s/step - accuracy: 0.5298 - loss: 5.5160 - val_accuracy: 0.6094 - val_loss: 7.8025
